In [ ]:
!pip install openai -q

## 환경 설정
1. openai 사이트에서 API KEY 발급
  
2. google colab - secret tap 에 `OPENAI_API_KEY`를 등록


In [5]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [6]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕 나는 조성렬이야 ~ 요즘 공부가 어려운데 공부방법좀~ "
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
response

ChatCompletion(id='chatcmpl-BlsA8OVEmKZ1N3oXJ4vKaJw2Wh2of', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요, 조성렬님! 공부가 어렵다고 느끼는 건 많은 사람들이 겪는 일이에요. 효과적인 공부 방법 몇 가지를 소개해드릴게요.\n\n1. **목표 설정**: 구체적이고 실현 가능한 목표를 세워보세요. 예를 들어, 하루에 몇 페이지 읽거나, 특정 개념을 이해하는 것을 목표로 할 수 있습니다.\n\n2. **시간 관리**: 공부할 시간을 정해 일정에 맞춰 공부하세요. Pomodoro 기법처럼 25분 동안 집중하고 5분 쉬는 방식을 사용해보는 것도 좋습니다.\n\n3. **적극적인 학습**: 단순히 읽거나 듣는 것을 넘어서 메모를 하거나 친구에게 설명해보세요. 내용을 다른 방식으로 표현해보는 것이 이해에 큰 도움이 됩니다.\n\n4. **복습**: 정기적으로 복습하는 것이 중요해요. 학습한 내용을 일정한 주기로 돌아보며 잊지 않도록 합니다.\n\n5. **환경 조성**: 집중할 수 있는 환경을 조성해보세요. 방해 요소를 최소화하고 조용한 공간에서 공부하는 것이 좋습니다.\n\n6. **건강 관리**: 좋은 식습관과 충분한 수면, 그리고 규칙적인 운동이 공부에 긍정적인 영향을 줄 수 있어요.\n\n7. **스트레스 관리**: 공부가 힘들 때는 잠시 쉬거나, 취미 활동을 하면서 스트레스를 풀어보세요.\n\n어때요? 도움이 될만한 방법이 있을까요? 더 구체적인 공부 방법이나 주제가 필요하면 말씀해 주세요!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1750749316, model='gpt-4o-mini-2024-07-18', object

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕 한성규! 졸린 이유는 여러 가지가 있을 수 있어. 예를 들어, 밤에 충분한 수면을 못 취했거나, 스트레스를 받고 있거나, 식사 후에 혈당이 급격히 올라갔다 떨어지면서 졸릴 수 있어. 또한, 환경적인 요인이나 단순히 몸이 피곤해서 그럴 수도 있어. 혹시 평소에 잠을 잘 자고 있어?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥정보 (어떤상황인지 상세히 풀어서)
3. Input Data/Example 입력, 예시 데이터
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현을 조정

- 프랑스 AFP 속보 시스템에서 도입되어 사용

In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: 과도한 업무로 끼니를 거르는 노동자들의 고충


## 함수 리팩토링

In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길~
- 교정 제목: "주말 폭우 예상, 모두 안전을 기원합니다"


## 연애 코치 ReAct

In [8]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [7]:
print(dating_coach('제가 어제 친구들이 놀다가 여자친구랑 연락을 잘 못해서, 단단히 삐져있는 상황입니다. 이 위기를 어떻게 극복해야 할까요?'))

1. 상황분석:
   당신의 여자친구는 연락이 잘 되지 않아서 섭섭함이나 불안을 느끼고 있을 가능성이 높습니다. 특히, 친구들과의 시간과 여자친구와의 시간의 균형을 유지하는 것이 중요한데, 이번에는 그 균형을 잃은 것 같습니다. 여기에 대한 그녀의 감정과 반응이 중요하며, 이는 신뢰와 소통의 문제로 확대될 수 있습니다.

2. 행동계획:
   - 먼저 여자친구의 감정을 이해하고 진심으로 사과해보세요. 연락이 잘 되지 않아서 그녀가 느꼈을 감정에 공감하는 것이 중요합니다.
   - 그녀와 대화할 시간을 정하고, 마음을 열고 솔직한 대화를 나누세요. 왜 연락을 못 했는지, 그 동안 당신의 생각과 느낀 감정도 공유하는 것이 좋습니다.
   - 앞으로 이러한 일이 발생하지 않도록 할 수 있는 방법을 논의하세요. 예를 들어, 친구들과의 시간을 보낼 때도 잠깐이라도 여자친구에게 연락을 한다거나, 중요한 일정은 미리 공유하는 것을 제안할 수 있습니다.

3. 실행:
   - 먼저, 당신이 그녀에게 연락을 시도하여 만남을 제안하세요. "내가 어제 너와 연락을 잘 못 해서 미안해. 너와 이야기하고 싶어."라는 메시지를 보내는 것도 좋습니다.
   - 직접 만나거나 전화로 대화하는 경우, 여기에 대한 사과와 함께 그녀의 감정을 물어보세요. "너는 어떻게 느꼈어? 내가 못 받은 것 같은 점이 있다면 정말 미안해."라는 식으로 접근해보세요.
   - 이후에는 약속한 내용에 따라 더욱 소통의 노력을 지속하세요. 자주 연락하고, 약속을 지키며 서로의 신뢰를 다시 쌓아가는 것이 중요합니다.


## 냉털마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Reasoning/Action을 끌어낼수 있는 적절한 프롬프팅을 작성하세요.


In [12]:
def fridge_recipe_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 시스템 역할 및 출력 형식 정의
    system_message = """
    << 시스템 지시사항 >>
    당신은 '냉장고 속 재료로 요리를 제안하는 코치'입니다. 사용자가 제공한 재료를 바탕으로,
    가장 합리적이고 창의적인 요리를 제안하세요. 단순한 재료 나열이 아닌,
    요리 선택의 이유와 단계별 조리 과정을 명확히 설명하세요.

    아래 형식에 맞춰 응답하세요:

    1. 상황 분석:
    - 사용자가 가진 재료의 종류 및 조합 특징 요약
    - 어떤 스타일의 요리가 가능할지 설명

    2. 행동 계획:
    - 만들 수 있는 요리 이름 제안 (1~2개)
    - 필요한 추가 재료가 있다면 명시

    3. 실행:
    - 추천 요리의 간단한 조리법 (단계별)
    - 난이도 및 예상 조리 시간

    4. 이유:
    - 왜 이 요리가 좋은 선택인지 설명
    - 영양, 맛, 재료, 칼로리 활용도 등 고려

    - **Input**:
        사용자의 냉장고에는 현재 [무, 파, 두부]이/가 있습니다.
    -   **Output**:
      1. 상황분석: 오늘은 냉장고에 있는 무, 파, 두부를 이용해 한식 스프를 만들어보겠습니다...
      2. 실행계획: 먼저 무와 파를 얇게 썰어 냄비에 넣고 물을 부어...
      3. 검증 및 추가내용 확인: 레시피를 확인한 결과, 무와 두부의 사용 방법에 대한 설명이 추가로 필요합니다...

    ※ 창의적이고 현실적인 조언을 우선합니다.
    """

    user_message = f"""
    사용자의 냉장고에는 현재 {prompt}이/가 있습니다.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={"type": "text"},
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [19]:
from IPython.display import Markdown

display(Markdown(fridge_recipe_coach('오징어 먹물, 흑마늘, 블루치즈, 유자청, 훈제 달걀, 트러플 오일, 악어고기')))

1. 상황 분석:
   - 사용자가 가진 재료들은 오징어 먹물, 흑마늘, 블루치즈, 유자청, 훈제 달걀, 트러플 오일, 악어고기입니다. 이 조합은 매우 독창적이고 이국적인 느낌을 주며, 특히 해산물과 육류의 조화가 돋보이는 요리를 시도할 수 있습니다.
   - 요리 스타일로는 현대적인 퓨전 요리나 고급 프랑스식 프리젠테이션을 추천하며, 특색 있는 소스와 조리 방법으로 색다른 경험을 선사할 수 있습니다.

2. 행동 계획:
   - 추천 요리 이름: '악어고기와 오징어 먹물 파스타' 혹은 '흑마늘 블루치즈 소스를 곁들인 훈제 달걀 샐러드'
   - 추가 재료: 파스타 면, 올리브 오일, 소금, 후추 (첫 번째 요리의 경우), 샐러드 채소 (두 번째 요리의 경우)

3. 실행:
   - **'악어고기와 오징어 먹물 파스타' 조리법**
     1. 악어고기는 먹기 좋게 썰어 소금과 후추로 간을 한다.
     2. 팬에 올리브 오일을 두르고 악어고기를 구워준다.
     3. 별도의 냄비에 물을 끓여 파스타 면을 삶는다 (알 덴테 상태).
     4. 팬에 오징어 먹물을 넣고, 삶은 파스타를 추가한다. 검은색 소스가 면에 잘 묻도록 섞는다.
     5. 마지막으로 구운 악어고기를 올리고, 트러플 오일로 마무리한다.
   - 난이도: 중간 / 예상 조리 시간: 30분

   - **'흑마늘 블루치즈 소스를 곁들인 훈제 달걀 샐러드' 조리법**
     1. 흑마늘을 잘게 다져서 블루치즈와 섞는다. 올리브 오일로 걸쭉한 소스를 만든다.
     2. 샐러드 채소(루꼴라, 시금치 등)를 그릇에 담는다.
     3. 훈제 달걀을 반으로 잘라 샐러드 위에 올린다.
     4. 흑마늘 블루치즈 소스를 샐러드에 뿌린 후, 유자청을 약간 곁들인다.
   - 난이도: 쉬움 / 예상 조리 시간: 15분

4. 이유:
   - 선택한 요리들은 사용자가 가진 독특한 재료를 최대한 활용할 수 있습니다. 특히 악어고기는 저지방 고단백으로 건강에 좋으며, 블루치즈와 흑마늘 소스는 풍미를 더해줘 미식가들의 입맛을 사로잡을 수 있습니다.
   - 두 요리 모두 색상, 맛, 텍스처의 조화가 뛰어나며 창의적인 디쉬로 손님을 대접할 때에도 좋습니다. 고급스러운 식사를 원하신다면 이 두 가지 요리 모두 추천드립니다.

## 면접 질문 생성 JSON 출력

In [21]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}
"""

    user_message = f"""
채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [22]:
job_posting = '''
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
'''

print(job_interview(job_posting))

{
    "hard_skill": [
        {
            "question": "Python을 사용하여 머신러닝 모델을 배포하는 과정에 대해 설명해 주시겠습니까?",
            "answer": "모델을 배포하기 위해 먼저 학습된 모델을 저장하고, Flask 또는 FastAPI와 같은 웹 프레임워크를 사용하여 API 서버를 구축합니다. 이후 Docker를 활용해 이를 컨테이너화하고, Kubernetes를 통해 클러스터의 여러 노드에 배포합니다. CI/CD 파이프라인을 Jenkins나 ArgoCD로 설정해 지속적으로 배포할 수 있도록 합니다."
        },
        {
            "question": "Docker와 Kubernetes의 차이점과 각각의 역할에 대해 설명해 주세요.",
            "answer": "Docker는 컨테이너 이미지를 만들고 배포하는 플랫폼이며, 응용 프로그램을 다른 환경에서 일관되게 실행할 수 있게 해줍니다. Kubernetes는 여러 개의 Docker 컨테이너를 관리하고 오케스트레이션하는 시스템으로, 장애 조치, 자동 배포, 수평적 확장 등을 지원합니다."
        },
        {
            "question": "MLflow를 사용하여 머신러닝 모델의 실험을 관리하는 방법에 대해 설명해 주시겠습니까?",
            "answer": "MLflow를 사용하면 실험을 쉽게 기록하고 관리할 수 있습니다. 모델 학습 시 다양한 하이퍼파라미터와 메트릭을 log하고, 최종 모델을 모델 레지스트리에 등록하여 관리할 수 있습니다. 서버를 통해 팀원들이 모델 버전 관리를 하고, 특정 모델을 쉽게 배포할 수 있게 도와줍니다."
        }
    ],
    "soft_skill": [
        {
            "question": "AI 모델 배포 시 알고리즘 개발자와의 협업에서 중요한 점은 무엇인가요?",
          

In [23]:
import json
output = job_interview(job_posting)
print(type(output))

# 응답데이터가 json 형식이 아니라면 파싱 오류 남
data = json.loads(output)
print(type(data))
print(data)

<class 'str'>
<class 'dict'>
{'hard_skill': [{'question': 'Python을 이용하여 AI 모델 배포를 위한 인프라를 구축할 때 어떤 주요 라이브러리를 사용할 것인가요?', 'answer': 'AI 모델 배포를 위해 Flask나 FastAPI와 같은 웹 프레임워크를 사용할 수 있습니다. 또한, TensorFlow Serving이나 TorchServe와 같은 라이브러리를 통해 모델을 더 쉽게 서빙할 수 있습니다. 이와 함께 Docker를 사용해 컨테이너화하고 Kubernetes를 통해 오케스트레이션할 수 있습니다.'}, {'question': 'Jenkins 기반의 CI/CD 파이프라인을 구축한 경험에 대해 설명해 주세요.', 'answer': 'Jenkins를 활용하여 코드가 변경될 때마다 자동으로 테스트와 빌드를 수행하도록 파이프라인을 설정했습니다. 이를 통해 개발 주기를 단축시키고, 코드 품질을 유지할 수 있었습니다. 또한, 테스트가 통과되면 자동으로 Docker 이미지를 빌드하고 Kubernetes 클러스터에 배포하는 설정을 추가했습니다.'}, {'question': '마이크로서비스를 Docker와 Kubernetes로 운영할 때의 주요 고려사항은 무엇인가요?', 'answer': '마이크로서비스 운영 시 고려해야 할 사항은 서비스 간의 네트워킹, 데이터베이스 연결, 로그 수집 및 모니터링입니다. 또한, 서비스의 스케일링과 롤링 업데이트를 어떻게 관리할지를 결정하는 것이 중요합니다. 또한, 보안 측면에서는 각 컨테이너의 권한 관리와 네트워크 정책 설정도 고려해야 합니다.'}], 'soft_skill': [{'question': '팀 내에서 알고리즘 개발자에게 기술 지원을 제공할 때 어떤 커뮤니케이션 방식을 채택하시겠습니까?', 'answer': '개발자와의 열린 커뮤니케이션을 유지하여, 문제점이나 요구사항을 신속하게 파악하는 것이 중요합니다. 주기적으로 미팅을 갖고, 개발자가 필요로 하는 기술적 지원을 명확히 이

In [24]:
hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']

print(hard_skill_qa)
print(soft_skill_qa)

[{'question': 'Python을 이용하여 AI 모델 배포를 위한 인프라를 구축할 때 어떤 주요 라이브러리를 사용할 것인가요?', 'answer': 'AI 모델 배포를 위해 Flask나 FastAPI와 같은 웹 프레임워크를 사용할 수 있습니다. 또한, TensorFlow Serving이나 TorchServe와 같은 라이브러리를 통해 모델을 더 쉽게 서빙할 수 있습니다. 이와 함께 Docker를 사용해 컨테이너화하고 Kubernetes를 통해 오케스트레이션할 수 있습니다.'}, {'question': 'Jenkins 기반의 CI/CD 파이프라인을 구축한 경험에 대해 설명해 주세요.', 'answer': 'Jenkins를 활용하여 코드가 변경될 때마다 자동으로 테스트와 빌드를 수행하도록 파이프라인을 설정했습니다. 이를 통해 개발 주기를 단축시키고, 코드 품질을 유지할 수 있었습니다. 또한, 테스트가 통과되면 자동으로 Docker 이미지를 빌드하고 Kubernetes 클러스터에 배포하는 설정을 추가했습니다.'}, {'question': '마이크로서비스를 Docker와 Kubernetes로 운영할 때의 주요 고려사항은 무엇인가요?', 'answer': '마이크로서비스 운영 시 고려해야 할 사항은 서비스 간의 네트워킹, 데이터베이스 연결, 로그 수집 및 모니터링입니다. 또한, 서비스의 스케일링과 롤링 업데이트를 어떻게 관리할지를 결정하는 것이 중요합니다. 또한, 보안 측면에서는 각 컨테이너의 권한 관리와 네트워크 정책 설정도 고려해야 합니다.'}]
[{'question': '팀 내에서 알고리즘 개발자에게 기술 지원을 제공할 때 어떤 커뮤니케이션 방식을 채택하시겠습니까?', 'answer': '개발자와의 열린 커뮤니케이션을 유지하여, 문제점이나 요구사항을 신속하게 파악하는 것이 중요합니다. 주기적으로 미팅을 갖고, 개발자가 필요로 하는 기술적 지원을 명확히 이해한 후, 그에 맞는 솔루션을 제공하려 노력합니다. 또한, 기술 지원 요청을 문서화하여 모두가 참고할 수 

## 다른 방법으로 해보기

In [25]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.
"""

    user_message = f"""

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}

채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [26]:
job_posting = '''
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
'''

print(job_interview(job_posting))

{
    "hard_skill": [
        {
            "question": "Python 프로그래밍 경험에 대해 설명해 주실 수 있나요?",
            "answer": "저는 Python을 활용한 다양한 프로젝트를 경험하였습니다. 예를 들어, 데이터 전처리 및 분석을 위해 Pandas와 NumPy를 사용했으며, 머신러닝 모델을 개발하기 위해 Scikit-learn과 TensorFlow를 활용했습니다. 이 과정에서 코드를 최적화하고, 효율적인 알고리즘을 구현하는 데 주력하였습니다."
        },
        {
            "question": "Docker와 Kubernetes를 활용하여 컨테이너 오케스트레이션을 수행한 경험은 무엇인가요?",
            "answer": "저는 Docker를 사용하여 여러 가지 애플리케이션을 컨테이너화하고, Kubernetes를 통해 클러스터를 구성하여 배포했습니다. 특히, Helm을 사용하여 배포를 자동화하고 여러 환경에서 연속적으로 테스트하는 CI/CD 파이프라인을 구축했습니다. 이러한 경험을 통해 운영 안정성을 크게 향상시킬 수 있었습니다."
        },
        {
            "question": "MLflow를 사용하여 모델의 학습 및 배포 과정을 관리한 경험이 있나요?",
            "answer": "네, MLflow를 활용하여 모델 학습, 기록 및 관리의 모든 과정을 통합했습니다. 프로젝트에서 각 모델의 하이퍼파라미터, 성과 지표 및 입력 데이터를 체계적으로 기록했으며, 이를 통해 모델의 버전 관리와 롤백을 쉽게 수행할 수 있었습니다. 이러한 과정을 통해 협업 시 투명성을 높이고 효율성을 높일 수 있었습니다."
        }
    ],
    "soft_skill": [
        {
            "question": "팀과의 효과적인 소통을 위해 어떤 방법을 사용하는지 설명해 주실 수 있나요?"